In [ ]:
!pip install nibabel
!pip install torchio
!pip install onnx
!pip install onnxruntime

In [2]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torchvision import datasets,transforms, models
import torchvision.transforms.functional as TF
import nibabel as nib
from pathlib import Path
import onnxruntime
from sklearn.metrics import precision_recall_fscore_support
from scipy import stats
import onnx

In [3]:
# to match fMRI ICA 100 components
batch_size = 100

In [4]:
class MultilayerPerceptron2(nn.Module):
  # whatever op layer is becomes num ip for next hidden layer
  def __init__(self, input_size=45*54*45, output_size=58):
    super().__init__()
    N = 200
    self.d1 = nn.Linear(input_size, N)
    self.d2 = nn.Linear(N, N)
    self.d3 = nn.Linear(N, N)
    self.d4 = nn.Linear(N, N)
    self.d5 = nn.Linear(N, output_size)
    self.dropout = nn.Dropout(0.66)
    self.flat = nn.Flatten()


  def forward(self,X):

    X = self.flat(X) #X.view(-1,45*54*45)
    X = F.relu(self.d1(X))
    X = F.relu(self.d3(X))
    X = self.dropout(X)
    X = F.relu(self.d4(X))
    X = self.d5(X)
    #X = torch.squeeze(X)

    return X

Download the mlp model weights from below link as provided in GitHub repo: https://github.com/AmmarPL/fMRI-Classification-JHU/ <br>
Model Weights - https://livejohnshopkins-my.sharepoint.com/:f:/g/personal/apallik1_jh_edu/Eqo4DojG33pBquC3_zCNYhYBSTcUS6Ppfhl9OAVF_erlZQ?e=bfn0E7

In [5]:
device = 'cuda' if torch.cuda.is_available() else "cpu"
print(f'Device is {device}')
model_final =  torch.load('/content/drive/MyDrive/Colab Notebooks/model_mlp_final.pth', map_location=device)

model = MultilayerPerceptron2()


model.load_state_dict(model_final['model'])
model = model.to(device)
model.eval()


Device is cpu


MultilayerPerceptron2(
  (d1): Linear(in_features=109350, out_features=200, bias=True)
  (d2): Linear(in_features=200, out_features=200, bias=True)
  (d3): Linear(in_features=200, out_features=200, bias=True)
  (d4): Linear(in_features=200, out_features=200, bias=True)
  (d5): Linear(in_features=200, out_features=58, bias=True)
  (dropout): Dropout(p=0.66, inplace=False)
  (flat): Flatten(start_dim=1, end_dim=-1)
)

Below cell saves the mlp model as an ONNX file which we can then use with OnnxBridge.

In [7]:
output_onnx = str(Path("mlp_model.onnx"))
input_volume = torch.randn(1, 1, 45, 54, 45)

# Export an ONNX model.
with torch.no_grad():
    torch.onnx.export(
        model=model,
        args=(input_volume),
        f=output_onnx,
        opset_version=13,
        verbose=True,
        input_names=["image"],
        output_names=[ "score"],
    )

Successfully saved mlp_model.onnx !!